### bash script to calculate KE
To find the files:  

REF08 = /data0/project/drakkar/CONFIGS/CREG12.L75/CREG12.L75-REF08-S/5d/  
FUT08 = /data0/project/drakkar/CONFIGS/CREG12.L75/CREG12.L75-FUT08-S/5d/  
WORK = /data0/project/drakkar/USERS/jrieck/  

And CDFTOOLS are here

CDF = /home/ctalandi/TOOLS/CDFTOOLS/bin

In [ ]:
#%%bash 

cd $REF08
for yyyy in {1996..2015}; do     
    echo ${yyyy};     
    for fileU in ${yyyy}/CREG12.L75-REF08_y${yyyy}*.5d_gridU.nc; do         
        fileV=${fileU/gridU/gridV};         
        fileT=${fileU/gridU/gridT};                
        fileK=${fileU/gridU/KE};         
        $CDF/cdfeke -u ${fileU} -v ${fileV} -t ${fileT} -mke -nc4 -o $WORK/CREG12.L75-REF08-KE/${fileK};       
        ncrename -O -v vomke,voke $WORK/CREG12.L75-REF08-KE/${fileK} $WORK/CREG12.L75-REF08-KE/${fileK};            
    done; 
done

cd $FUT08
for yyyy in {2051..2070}; do     
    echo ${yyyy};     
    for fileU in ${yyyy}/CREG12.L75-FUT08_y${yyyy}*.5d_gridU.nc; do         
        fileV=${fileU/gridU/gridV};         
        fileT=${fileU/gridU/gridT};               
        fileK=${fileU/gridU/KE};         
        $CDF/cdfeke -u ${fileU} -v ${fileV} -t ${fileT} -mke -nc4 -o $WORK/CREG12.L75-FUT08-KE/${fileK};       
        ncrename -O -v vomke,voke $WORK/CREG12.L75-FUT08-KE/${fileK} $WORK/CREG12.L75-FUT08-KE/${fileK};     
    done; 
done

CDFTOOLS to calculate Richardson number

In [ ]:
#%%bash

cd $WORK
for yyyy in {1996..2015}; do     
    echo ${yyyy};     
    for fileU in $REF08/${yyyy}/CREG12.L75-REF08_y${yyyy}*.5d_gridU.nc; do         
        fileV=${fileU/gridU/gridV};         
        fileT=${fileU/gridU/gridT};         
        fileR=$(echo ${fileU/gridU/RI} | rev | cut -d'/' -f-2 | rev);                 
        $CDF/cdfrichardson -t ${fileT} -u ${fileU} -v ${fileV} -o $WORK/CREG12.L75-REF08-RI/${fileR};              
    done; 
done

cd $WORK
for yyyy in {2051..2059}; do     
    echo ${yyyy};     
    for fileU in $FUT08/${yyyy}/CREG12.L75-FUT08_y${yyyy}*.5d_gridU.nc; do         
        fileV=${fileU/gridU/gridV};         
        fileT=${fileU/gridU/gridT};         
        fileR=$(echo ${fileU/gridU/RI} | rev | cut -d'/' -f-2 | rev);                 
        $CDF/cdfrichardson -t ${fileT} -u ${fileU} -v ${fileV} -o $WORK/CREG12.L75-FUT08-RI/${fileR};              
    done; 
done

### calculate climatological seasonal cycle of U and V

In [ ]:
cd $WORK/CREG12.L75-REF08-UV_clim
for fivedstep in {0..72}; 
    do step=`printf "%02d" ${fivedstep}`
    echo ${step}; 
    ncra -O -4 -v vozocrtx -d time_counter,${fivedstep},,73 $REF08/199[6-9]/CREG12.L75-REF08_y*.5d_gridU.nc $REF08/20??/CREG12.L75-REF08_y*.5d_gridU.nc U_tmp${step}.nc
    ncra -O -4 -v vomecrty -d time_counter,${fivedstep},,73 $REF08/199[6-9]/CREG12.L75-REF08_y*.5d_gridV.nc $REF08/20??/CREG12.L75-REF08_y*.5d_gridV.nc V_tmp${step}.nc
done
ncrcat -O -4 U_tmp??.nc CREG12.L75-REF08_1996-2015_climatology_gridU.nc
ncrcat -O -4 V_tmp??.nc CREG12.L75-REF08_1996-2015_climatology_gridV.nc

cd $WORK/CREG12.L75-FUT08-UV_clim
for fivedstep in {0..72}; 
    do step=`printf "%02d" ${fivedstep}`
    echo ${step}; 
    ncra -O -4 -v vozocrtx -d time_counter,${fivedstep},,73 $FUT08/????/CREG12.L75-FUT08_y*.5d_gridU.nc U_tmp${step}.nc
    ncra -O -4 -v vomecrty -d time_counter,${fivedstep},,73 $FUT08/????/CREG12.L75-FUT08_y*.5d_gridV.nc V_tmp${step}.nc
done
ncrcat -O -4 U_tmp??.nc CREG12.L75-FUT08_2051-2070_climatology_gridU.nc
ncrcat -O -4 V_tmp??.nc CREG12.L75-FUT08_2051-2070_climatology_gridV.nc

In [ ]:
cd $WORK/CREG12.L75-FUT08-UV_clim
k=0
for fileR in /data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-RI/2063/CREG12.L75-FUT08_y*.5d_RI.nc;
    do K=`printf "%02d" ${k}`
    tmp=$(echo ${fileR} | rev | cut -d'/' -f-1 | rev)
    tmp2=$(echo ${tmp/RI/gridUclim})
    fileU=${tmp2/y2063/}
    fileV=${fileU/gridUclim/gridVclim}
    mv U_tmp${K}.nc ${fileU}
    mv V_tmp${K}.nc ${fileV}
    k=$(($k+1))
done

In [ ]:
cd $WORK/CREG12.L75-REF08-UV_clim
k=0
for fileR in /data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-RI/2001/CREG12.L75-REF08_y*.5d_RI.nc;
    do K=`printf "%02d" ${k}`
    tmp=$(echo ${fileR} | rev | cut -d'/' -f-1 | rev)
    tmp2=$(echo ${tmp/RI/gridUclim})
    fileU=${tmp2/y2001/}
    fileV=${fileU/gridUclim/gridVclim}
    mv U_tmp${K}.nc ${fileU}
    mv V_tmp${K}.nc ${fileV}
    k=$(($k+1))
done

In [ ]:
cd $WORK/CREG12.L75-REF08-UV_clim
k=0
for fileU in CREG12.L75-REF08_m*.5d_gridUclim.nc;
    do K=`printf "%02d" ${k}`
    fileV=${fileU/gridUclim/gridVclim}
    cp ${fileU} U_tmp${K}.nc
    cp ${fileV} V_tmp${K}.nc
    k=$(($k+1))
done

### calculate EKE and MKE from anomalies to the climatlogy

In [ ]:
cd $WORK
for yyyy in {1996..2015}; do     
    echo ${yyyy};     
    for fileU in $REF08/${yyyy}/CREG12.L75-REF08_y${yyyy}*.5d_gridU.nc; do         
        fileV=${fileU/gridU/gridV};
        fileT=${fileU/gridU/gridT}; 
        fileUmTMP=$(echo ${fileU/gridU/gridUclim} | rev | cut -d'/' -f-1 | rev)
        fileVmTMP=$(echo ${fileV/gridV/gridVclim} | rev | cut -d'/' -f-1 | rev)
        fileUm=CREG12.L75-REF08-UV_clim/CREG12.L75-REF08_m$(echo $fileUmTMP | rev | cut -d'm' -f-2 | rev)
        fileVm=CREG12.L75-REF08-UV_clim/CREG12.L75-REF08_m$(echo $fileVmTMP | rev | cut -d'm' -f-2 | rev)
        fileE=$(echo ${fileU/gridU/EKE} | rev | cut -d'/' -f-2 | rev);
        ncrename -O -v vozocrtx,vozocrtxMEAN ${fileUm} tmpU.nc
        ncrename -O -v vomecrty,vomecrtyMEAN ${fileVm} tmpV.nc
        ncks -A -v vozocrtx -4 ${fileU} tmpU.nc
        ncks -A -v vomecrty -4 ${fileV} tmpV.nc
        ncap2 -O -4 -s 'vozocrtxPRIME=vozocrtx-vozocrtxMEAN' tmpU.nc tmpU.nc
        ncap2 -O -4 -s 'vomecrtyPRIME=vomecrty-vomecrtyMEAN' tmpV.nc tmpV.nc
        ncks -O -4 -v vozocrtxPRIME tmpU.nc tmpU.nc
        ncks -O -4 -v vomecrtyPRIME tmpV.nc tmpV.nc
        ncrename -O -v vozocrtxPRIME,vozocrtx tmpU.nc tmpU.nc
        ncrename -O -v vomecrtyPRIME,vomecrty tmpV.nc tmpV.nc
        $CDF/cdfeke -u tmpU.nc -v tmpV.nc -t ${fileT} -mke -nc4 -o CREG12.L75-REF08-EKE/${fileE}
        ncrename -O -v .vomke,voeke CREG12.L75-REF08-EKE/${fileE} CREG12.L75-REF08-EKE/${fileE}
    done; 
done
cd CREG12.L75-REF08-UV_clim
fileT=$REF08/1996/CREG12.L75-REF08_y1996m01d05.5d_gridT.nc
for fileUm in CREG12.L75-REF08_m*gridUclim.nc; do
    fileVm=${fileUm/gridU/gridV};
    fileM=${fileUm/gridUclim/MKE}
    $CDF/cdfeke -u ${fileUm} -v ${fileVm} -t ${fileT} -mke -nc4 -o ../CREG12.L75-REF08-MKE/${fileM}
done

In [ ]:
cd $WORK
for yyyy in {2051..2070}; do     
    echo ${yyyy};     
    for fileU in $FUT08/${yyyy}/CREG12.L75-FUT08_y${yyyy}*.5d_gridU.nc; do         
        fileV=${fileU/gridU/gridV};
        fileT=${fileU/gridU/gridT}; 
        fileUmTMP=$(echo ${fileU/gridU/gridUclim} | rev | cut -d'/' -f-1 | rev)
        fileVmTMP=$(echo ${fileV/gridV/gridVclim} | rev | cut -d'/' -f-1 | rev)
        fileUm=CREG12.L75-FUT08-UV_clim/CREG12.L75-FUT08_m$(echo $fileUmTMP | rev | cut -d'm' -f-2 | rev)
        fileVm=CREG12.L75-FUT08-UV_clim/CREG12.L75-FUT08_m$(echo $fileVmTMP | rev | cut -d'm' -f-2 | rev)
        fileE=$(echo ${fileU/gridU/EKE} | rev | cut -d'/' -f-2 | rev);
        ncrename -O -v vozocrtx,vozocrtxMEAN ${fileUm} futtmpU.nc
        ncrename -O -v vomecrty,vomecrtyMEAN ${fileVm} futtmpV.nc
        ncks -A -v vozocrtx -4 ${fileU} futtmpU.nc
        ncks -A -v vomecrty -4 ${fileV} futtmpV.nc
        ncap2 -O -4 -s 'vozocrtxPRIME=vozocrtx-vozocrtxMEAN' futtmpU.nc futtmpU.nc
        ncap2 -O -4 -s 'vomecrtyPRIME=vomecrty-vomecrtyMEAN' futtmpV.nc futtmpV.nc
        ncks -O -4 -v vozocrtxPRIME futtmpU.nc futtmpU.nc
        ncks -O -4 -v vomecrtyPRIME futtmpV.nc futtmpV.nc
        ncrename -O -v vozocrtxPRIME,vozocrtx futtmpU.nc futtmpU.nc
        ncrename -O -v vomecrtyPRIME,vomecrty futtmpV.nc futtmpV.nc
        $CDF/cdfeke -u futtmpU.nc -v futtmpV.nc -t ${fileT} -mke -nc4 -o CREG12.L75-FUT08-EKE/${fileE}
        ncrename -O -v .vomke,voeke CREG12.L75-FUT08-EKE/${fileE} CREG12.L75-FUT08-EKE/${fileE}
    done; 
done
cd CREG12.L75-FUT08-UV_clim
fileT=$FUT08/2051/CREG12.L75-FUT08_y2051m01d05.5d_gridT.nc
for fileUm in CREG12.L75-FUT08_m*gridUclim.nc; do
    fileVm=${fileUm/gridU/gridV};
    fileM=${fileUm/gridUclim/MKE}
    $CDF/cdfeke -u ${fileUm} -v ${fileVm} -t ${fileT} -mke -nc4 -o ../CREG12.L75-FUT08-MKE/${fileM}
done

test using xarray for extraction (was not very efficient)

In [ ]:
CONF = CONFS[1]
gridT_files = glob("/data0/project/drakkar/CONFIGS/CREG12.L75/" + CONF + "-S/5d/206?/" + CONF 
                   + "_*.5d_gridT.nc") + glob("/data0/project/drakkar/CONFIGS/CREG12.L75/" + CONF + "-S/5d/2070/" + CONF + "_*.5d_gridT.nc")
icemod_files = glob("/data0/project/drakkar/CONFIGS/CREG12.L75/" + CONF + "-S/5d/206?/" + CONF 
                   + "_*.5d_icemod.nc") + glob("/data0/project/drakkar/CONFIGS/CREG12.L75/" + CONF + "-S/5d/2070/" + CONF + "_*.5d_icemod.nc")
KE_files = glob("/data0/project/drakkar/USERS/jrieck/" + CONF + "-KE/206?/" + CONF 
                + "_*.5d_KE.nc") + glob("/data0/project/drakkar/USERS/jrieck/" + CONF + "-KE/2070/" + CONF + "_*.5d_KE.nc")
RI_files = glob("/data0/project/drakkar/USERS/jrieck/" + CONF + "-RI/206?/" + CONF 
                + "_*.5d_RI.nc") + glob("/data0/project/drakkar/USERS/jrieck/" + CONF + "-RI/2070/" + CONF + "_*.5d_RI.nc")

for i in np.arange(0, len(moorn)):
    # seems like the extraction of profiles is a bit faster when selecting the ij-point already during the preprocessing
    def preproT(ds):
        ds = ds.isel(x=moorx[i], y=moory[i])
        return ds.drop(["mldkz5", "botpres", "time_centered_bounds", "time_counter_bounds", "time_centered", "deptht_bounds"])
    def preproI(ds):
        ds = ds.isel(x=moorx[i], y=moory[i])
        return ds.drop(["sitemp", "sisali", "siages", "snthic", "snvolu", "iicestru", "iicestrv", "sivelo", "sivolu", "sistre", 
                        "sidive", "sishea", "intstrx", "intstry", "time_centered_bounds", "time_counter_bounds", "time_centered"])
    def preproK(ds):
        ds = ds.isel(x=moorx[i], y=moory[i])
        return ds.set_coords(("nav_lat", "nav_lon"))
    def preproR(ds):
        ds = ds.isel(x=moorx[i], y=moory[i])
        return ds.set_coords(("nav_lat", "nav_lon"))
    moorI = f'{moorx[i]:04}'
    moorJ = f'{moory[i]:04}'
    gridT = xr.decode_cf(xr.open_mfdataset(gridT_files, chunks={"time_counter": 73, "deptht": 75}, preprocess=preproT, 
                                           decode_cf=False, coords="minimal", compat="override", parallel=True))
    icemod = xr.decode_cf(xr.open_mfdataset(icemod_files, chunks={"time_counter": 73}, preprocess=preproI, 
                                            decode_cf=False, coords="minimal", compat="override", parallel=True))
    KE = xr.decode_cf(xr.open_mfdataset(KE_files, chunks={"time_counter": 73, "deptht": 75}, preprocess=preproK, 
                                        decode_cf=False, coords="minimal", compat="override", parallel=True))
    RI = xr.decode_cf(xr.open_mfdataset(RI_files, chunks={"time_counter": 73, "deptht": 75}, preprocess=preproR, 
                                        decode_cf=False, coords="minimal", compat="override", parallel=True))
    ds = xr.merge([gridT, icemod, KE, RI], compat="override").sel(time_counter=slice(str(years[CONF][0]) + "-01-01", str(years[CONF][-1]) + "-12-31"))
    years, datasets = zip(*ds.groupby("time_counter.year"))
    paths = [str("/data0/project/drakkar/USERS/jrieck/" + CONF + "-moorings/" + CONF + "_" + str(y) + "_moor" + moorn[i] + "_I" + moorI + "J" + moorJ 
             + "_gridT.nc") for y in years]
    xr.save_mfdataset(datasets, paths, engine="netcdf4", format="NETCDF4")
        

### bash scripts to extract points

This is slow as well. Using CDFTOOLS to extract from the 3D fields and ncks to extract from 2D fields. Not sure whether ncks could be faster for the 3D fields as well?

In [ ]:
#I=540
#J=1530
#number=001
#cd $REF08
#for year in {1996..2015}; do
#    echo $year
#    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_gridT.nc; do 
#        $CDF/cdfprofile -f $file -v votemper -IJ $I $J -o $WORK/CREG12.L75-REF08-moorings/$file.votemper.profile; done
#    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_gridT.nc; do 
#        $CDF/cdfprofile -f $file -v vosaline -IJ $I $J -o $WORK/CREG12.L75-REF08-moorings/$file.vosaline.profile; done
#    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_gridT.nc; do 
#        $CDF/cdfprofile -f $file -v rhop_sig0 -IJ $I $J -o $WORK/CREG12.L75-REF08-moorings/$file.rhop_sig0.profile; done
#done
#cd $WORK/CREG12.L75-REF08-moorings/
#ncrcat -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 ????/*gridT.nc.votemper.profile CREG12.L75-REF08_y1996-2015_moor${number}_I0${I}J${J}_gridT_votemper.nc
#ncrcat -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 ????/*gridT.nc.vosaline.profile CREG12.L75-REF08_y1996-2015_moor${number}_I0${I}J${J}_gridT_vosaline.nc
#ncrcat -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 ????/*gridT.nc.rhop_sig0.profile CREG12.L75-REF08_y1996-2015_moor${number}_I0${I}J${J}_gridT_rhop_sig0.nc
#
#I=540
#J=1530
#number=002
#cd $REF08
#for year in {1996..2015}; do
#    echo $year
#    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_gridT.nc; do 
#        ncks -O -d x,$I -d y,$J -v mldr10_1 $file $WORK/CREG12.L75-REF08-moorings/$file.mldr10_1.profile; done
#    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_icemod.nc; do 
#        ncks -O -d x,$I -d y,$J -v siconc $file $WORK/CREG12.L75-REF08-moorings/$file.siconc.profile; done
#    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_icemod.nc; do 
#        ncks -O -d x,$I -d y,$J -v sithic $file $WORK/CREG12.L75-REF08-moorings/$file.sithic.profile; done
#done
#cd $WORK/CREG12.L75-REF08-moorings/
#ncrcat -4 --cnk_dmn time_counter,1460 ????/*gridT.nc.mldr10_1.profile CREG12.L75-REF08_y1996-2015_moor${number}_I0${I}J${J}_gridT_mldr10_1.nc
#ncrcat -4 --cnk_dmn time_counter,1460 ????/*icemod.nc.siconc.profile CREG12.L75-REF08_y1996-2015_moor${number}_I0${I}J${J}_gridT_siconc.nc
#ncrcat -4 --cnk_dmn time_counter,1460 ????/*icemod.nc.sithic.profile CREG12.L75-REF08_y1996-2015_moor${number}_I0${I}J${J}_gridT_sithic.nc
#
#I=540
#J=1530
#number=001
#cd $WORK/CREG12.L75-REF08-KE
#for year in {1996..2015}; do
#    echo $year
#    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_KE.nc; do 
#        $CDF/cdfprofile -f $file -v voke -IJ $I $J -o $WORK/CREG12.L75-REF08-moorings/$file.voke.profile; done
#done
#
#ncrcat -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 ????/*.nc.voke.profile CREG12.L75-REF08_y1996-2015_moor${number}_I0${I}J${J}_gridT_voke.nc

Using only ncks seems to work just fine. This way I can extract several variables at once. This takes a while but if I start extraction of each mooring point in parallel, it can be done overnight for one simulation.

In [ ]:
I=540
J=1530
number=001
for year in {1996..2015}; do
    echo $year
    cd $REF08
    echo "gridT"
    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_gridT.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v votemper,vosaline,rhop_sig0,mldr10_1 $file $WORK/CREG12.L75-REF08-moorings/$file.votemper.profile.I0${I}J${J}; done
    echo "icemod"
    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_icemod.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v siconc,sithic $file $WORK/CREG12.L75-REF08-moorings/$file.icemod.profile.I0${I}J${J}; done
    cd $WORK/CREG12.L75-REF08-RI
    echo "voric"
    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_RI.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v voric $file $WORK/CREG12.L75-REF08-moorings/$file.voric.profile.I0${I}J${J}; done
done

re-extract voke later because I recompute it right now

In [ ]:
I=540
J=1530
number=001
for year in {1996..2015}; do
    echo $year
    cd $WORK/CREG12.L75-REF08-KE
    echo "voke"
    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_KE.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v voke $file $WORK/CREG12.L75-REF08-moorings/$file.voke.profile.I0${I}J${J}; done
done

In [ ]:
I=540
J=1530
number=001
cd $WORK/CREG12.L75-REF08-moorings/
ncrcat -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 ????/*.nc.voke.profile.I0${I}J${J} CREG12.L75-REF08_y1996-2015_moor${number}_I${I}J${J}_gridT_voke.nc
ncrcat -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 ????/*gridT.nc.votemper.profile.I0${I}J${J} CREG12.L75-REF08_y1996-2015_moor${number}_I${I}J${J}_gridT.nc
ncrcat -4 --cnk_dmn time_counter,1460 ????/*icemod.nc.icemod.profile.I0${I}J${J} CREG12.L75-REF08_y1996-2015_moor${number}_I${I}J${J}_icemod.nc
ncrcat -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 ????/*.nc.voric.profile.I0${I}J${J} CREG12.L75-REF08_y1996-2015_moor${number}_I${I}J${J}_gridT_voric.nc

In [ ]:
I=540
J=1530
number=001
for year in {2051..2070}; do
    echo $year
    cd $FUT08
    echo "gridT"
    for file in ${year}/CREG12.L75-FUT08_y${year}m*.5d_gridT.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v votemper,vosaline,rhop_sig0,mldr10_1 $file $WORK/CREG12.L75-FUT08-moorings/$file.votemper.profile.I0${I}J${J}; done
    echo "icemod"
    for file in ${year}/CREG12.L75-FUT08_y${year}m*.5d_icemod.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v siconc,sithic $file $WORK/CREG12.L75-FUT08-moorings/$file.icemod.profile.I0${I}J${J}; done
    cd $WORK/CREG12.L75-FUT08-RI
    echo "voric"
    for file in ${year}/CREG12.L75-FUT08_y${year}m*.5d_RI.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v voric $file $WORK/CREG12.L75-FUT08-moorings/$file.voric.profile.I0${I}J${J}; done
done

re-extract voke later because I recompute it right now

In [ ]:
I=540
J=1530
number=001
for year in {2051..2070}; do
    echo $year
    cd $WORK/CREG12.L75-FUT08-KE
    echo "voke"
    for file in ${year}/CREG12.L75-FUT08_y${year}m*.5d_KE.nc; do
        ncks -O -4 -d x,$I -d y,$J -v voke $file $WORK/CREG12.L75-FUT08-moorings/$file.voke.profile.I0${I}J${J}; done
done

In [ ]:
I=540
J=1530
number=001
cd $WORK/CREG12.L75-FUT08-moorings/
ncrcat -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 ????/*.nc.voke.profile.I0${I}J${J} CREG12.L75-FUT08_y2051-2070_moor${number}_I${I}J${J}_gridT_voke.nc
ncrcat -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 ????/*gridT.nc.votemper.profile.I0${I}J${J} CREG12.L75-FUT08_y2051-2070_moor${number}_I${I}J${J}_gridT.nc
ncrcat -4 --cnk_dmn time_counter,1460 ????/*icemod.nc.icemod.profile.I0${I}J${J} CREG12.L75-FUT08_y2051-2070_moor${number}_I${I}J${J}_icemod.nc
ncrcat -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 ????/*.nc.voric.profile.I0${I}J${J} CREG12.L75-FUT08_y2051-2070_moor${number}_I${I}J${J}_gridT_voric.nc

### extract moorings of EKE and MKE

In [ ]:
I=540
J=1530
number=001

I=820
J=1250
number=002

I=960
J=1150
number=003

I=650
J=1220
number=004

I=1040
J=1320
number=005

I=840
J=1540
number=006

I=870
J=1660
number=007

I=1200
J=1000
number=008

I=660
J=1550
number=009

I=540
J=1620
number=WAL

I=1080
J=1460
number=WLS

I=1070
J=1400
number=WSZ

In [ ]:
I=540
J=1530
number=001
for year in {1996..2015}; do
    echo $year
    cd $WORK/CREG12.L75-REF08-EKE
    echo "voeke"
    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_EKE.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v voeke $file $WORK/CREG12.L75-REF08-moorings/$file.voeke.profile.I0${I}J${J}; done
done
cd $WORK/CREG12.L75-REF08-MKE
echo "vomke"
for file in CREG12.L75-REF08_m*.5d_MKE.nc; do 
    ncks -O -4 -d x,$I -d y,$J -v vomke $file $WORK/CREG12.L75-REF08-moorings/MKE/$file.vomke.profile.I0${I}J${J}; done

In [ ]:
cd $WORK/CREG12.L75-REF08-moorings/
ncrcat -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 ????/*.nc.voeke.profile.I0${I}J${J} CREG12.L75-REF08_y1996-2015_moor${number}_I${I}J${J}_gridT_voeke.nc
ncrcat -4 --cnk_dmn time_counter,73 --cnk_dmn deptht,75 MKE/*.nc.vomke.profile.I0${I}J${J} CREG12.L75-REF08_y1996-2015_moor${number}_I${I}J${J}_gridT_vomke.nc

In [ ]:
I=540
J=1530
number=001
for year in {2051..2070}; do
    echo $year
    cd $WORK/CREG12.L75-FUT08-EKE
    echo "voeke"
    for file in ${year}/CREG12.L75-FUT08_y${year}m*.5d_EKE.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v voeke $file $WORK/CREG12.L75-FUT08-moorings/$file.voeke.profile.I0${I}J${J}; done
done
cd $WORK/CREG12.L75-FUT08-MKE
echo "vomke"
for file in CREG12.L75-FUT08_m*.5d_MKE.nc; do 
    ncks -O -4 -d x,$I -d y,$J -v vomke $file $WORK/CREG12.L75-FUT08-moorings/MKE/$file.vomke.profile.I0${I}J${J}; done

In [ ]:
cd $WORK/CREG12.L75-FUT08-moorings/
ncrcat -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 ????/*.nc.voeke.profile.I0${I}J${J} CREG12.L75-FUT08_y2051-2070_moor${number}_I${I}J${J}_gridT_voeke.nc
ncrcat -4 --cnk_dmn time_counter,73 --cnk_dmn deptht,75 MKE/*.nc.vomke.profile.I0${I}J${J} CREG12.L75-FUT08_y2051-2070_moor${number}_I${I}J${J}_gridT_vomke.nc

### End of preparations
---